Downloads the Tiny Shakespeare dataset from a URL using `tf.keras.utils.get_file` and reads it into a string variable `shakespeare_text`.

In [ ]:
import tensorflow as tf

shakespear_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt",shakespear_url)
with open(filepath) as f:
  shakespeare_text = f.read()

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Prints the first 80 characters of the downloaded Shakespeare text to get a preview of the data.

In [ ]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


Initializes a `TextVectorization` layer for character-level tokenization, adapts it to the Shakespeare text to build the vocabulary, and then encodes the text into a sequence of character IDs.

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

Adjusts the encoded character IDs by subtracting 2 and calculates the number of unique tokens in the vocabulary after the adjustment.

In [ ]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() -2

Defines a function `to_dataset` that takes a sequence of character IDs and converts it into a TensorFlow Dataset of input/target window pairs for training a sequence model.

In [ ]:
def to_dataset(sequence,length,shuffle=False,seed=None,batch_size=32):
  ds = tf.data.Dataset.from_tensor_slices(sequence)
  ds = ds.window(length + 1,shift=1,drop_remainder=True)
  ds = ds.flat_map(lambda window_ds:window_ds.batch(length+1))
  if shuffle:
    ds = ds.shuffle(buffer_size=100_00,seed=seed)
  ds = ds.batch(batch_size)
  return ds.map(lambda window: (window[:,:-1],window[:,1:])).prefetch(1)

Sets the window length for the dataset and splits the encoded text into training, validation, and test sets using the `to_dataset` function with specified sizes and shuffling for the training set.

In [ ]:
length =100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000],length=length,shuffle=True,seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000],length=length)
test_set = to_dataset(encoded[1_060_000:],length=length)

Builds a character-level RNN model using a GRU layer for sequence processing, followed by a Dense layer with softmax activation for character prediction. Compiles the model with sparse categorical crossentropy loss and Nadam optimizer, and sets up a ModelCheckpoint callback to save the best model based on validation accuracy during training.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens,output_dim=16),
    tf.keras.layers.GRU(128,return_sequences=True),
    tf.keras.layers.Dense(n_tokens,activation="softmax")
])

model.compile(loss='sparse_categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])

# model_ckpt = tf.keras.callbacks.ModelCheckpoint(
#     "shakespeare_model.keras",monitor ="val_accuracy",save_best_only=True
# )

# history = model.fit(train_set,validation_data=valid_set,epochs=10,callbacks=[model_ckpt])

# As training takes a lot of time, we are using the pretained model of shakespeare RNN model

In [ ]:
import pathlib
import tensorflow as tf



In [ ]:

shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

In [ ]:
url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = tf.keras.utils.get_file("shakespeare_model.tgz",url,extract=True)
model_path = pathlib.Path(path).with_suffix('').joinpath('shakespeare_model')

352865/352865 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model_path

PosixPath('/root/.keras/datasets/shakespeare_model_extracted/shakespeare_model')

In [ ]:
loaded_model = tf.keras.Sequential([
    tf.keras.layers.TFSMLayer(
        model_path,
        call_endpoint='serving_default'
    )
])

In [ ]:
loaded = tf.saved_model.load(model_path)


In [ ]:
print("Available signatures:", list(loaded.signatures.keys()))


Available signatures: ['serving_default']


In [ ]:
infer = None
if "serving_default" in loaded.signatures:
    infer = loaded.signatures["serving_default"]
else:
    # fallback to direct call
    infer = loaded.__call__

In [ ]:
inputs = tf.keras.Input(shape=(), dtype=tf.string)  # say input is a string


In [ ]:
x = text_vec_layer(inputs)          # shape (batch, seq_len)


In [ ]:
x = tf.keras.layers.Lambda(lambda X: X - 2)(x)


In [ ]:
prompt = tf.constant(["To be or not to b"])
# Pass the raw string prompt directly to the infer function
outputs = infer(text_vectorization_input=prompt)
logits = outputs['sequential']  # or outputs['output_0'] depending on signature

In [ ]:
logits


<tf.Tensor: shape=(1, 17, 39), dtype=float32, numpy=
array([[[1.81564420e-01, 6.64177835e-02, 6.29598368e-03, 1.06545582e-01,
         2.56400704e-02, 7.74683729e-02, 3.66694510e-01, 4.69604880e-03,
         3.51578742e-02, 1.17589188e-05, 9.23981145e-03, 1.84964836e-02,
         5.56145551e-07, 1.79435108e-02, 2.96746759e-04, 6.57376461e-03,
         1.29411472e-02, 2.60792095e-02, 1.12853968e-03, 9.53492636e-05,
         1.11382235e-06, 1.23063161e-04, 1.17037678e-03, 9.44407936e-03,
         2.98096296e-07, 3.17247277e-08, 1.23338113e-02, 5.36633283e-03,
         1.70396850e-03, 3.56346485e-03, 1.34389906e-03, 1.66126655e-03,
         5.96528480e-07, 1.22643706e-09, 1.27851008e-09, 1.44478605e-07,
         6.61341926e-10, 4.58126641e-19, 9.39734601e-09],
        [7.48781800e-01, 8.53533311e-06, 1.50150327e-05, 1.44324657e-02,
         8.39594850e-06, 7.65351229e-04, 2.05993449e-04, 6.82524929e-04,
         7.88733885e-02, 2.04552896e-02, 1.16906092e-02, 1.67817809e-02,
         1.00

In [ ]:
# Get predicted token ID (Tensor)
y_pred = tf.argmax(logits[0, -1])  # logits shape: [1, seq_len, vocab_size]

# Convert to Python int
pred_id = y_pred.numpy()

# Access vocabulary safely
vocab = text_vec_layer.get_vocabulary()
predicted_char = vocab[pred_id + 2]  # Only if you know +2 is correct
print(predicted_char)


e


Now that we have the loaded model and can predict the next character, we can write a function to generate a sequence of text.

In [ ]:
log_probas = tf.math.log([.5,.5,.1])
log_probas

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.6931472, -0.6931472, -2.3025851], dtype=float32)>

In [ ]:
tf.random.set_seed(42)
tf.random.categorical(tf.reshape(log_probas, [1, -1]), num_samples=8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 1, 0, 2, 1, 1, 0, 1]])>

In [ ]:
def next_char(text,temperature=1):
  outputs = infer(text_vectorization_input=tf.constant([text]))
  logits = outputs['sequential'][:, -1, :] # Select logits for the last character
  rescaled_logits = logits / temperature # Use logits directly for temperature scaling
  char_id = tf.random.categorical(rescaled_logits,num_samples=1)[0,0]
  return text_vec_layer.get_vocabulary()[char_id.numpy()+2] # Convert char_id to numpy scalar

In [ ]:
def generate_text(text, n_chars=200, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
tf.random.set_seed(42)
print(generate_text("To be or not to be",temperature=0.01))

To be or not to be the duke
as it is a proper and the duke is a proper and the contents
as the strange provost, and the duke of your honour.

provost:
i will not be a strange daughter is a strange daughter
to the death


In [ ]:
!pip install gradio -q
import gradio as gr

def gradio_generate_text(prompt, n_chars=200, temperature=1):
    # Reuse the existing generate_text function
    return generate_text(prompt, n_chars, temperature)

# Define the Gradio interface
interface = gr.Interface(
    fn=gradio_generate_text,
    inputs=[
        gr.Textbox(label="Prompt", value="To be or not to be"),
        gr.Slider(minimum=50, maximum=500, value=200, label="Number of characters to generate"),
        gr.Slider(minimum=0.01, maximum=2.0, value=1.0, label="Temperature")
    ],
    outputs=gr.Textbox(label="Generated Text", lines=10), # Added lines parameter to increase height
    title="Shakespearean Text Generator",
    description="Generate text in the style of Shakespeare using a trained RNN model."
)

# Launch the interface
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5209335bc6b37e3cab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
